# Spark + SciPy + Matemáticas para ML

## Objetivos
- Operaciones matemáticas avanzadas con SciPy
- Optimización distribuida
- Estadística y análisis numérico
- Álgebra lineal a escala

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import pandas_udf, col
from pyspark.sql.types import FloatType, ArrayType

import scipy
from scipy import optimize, stats, linalg, special, integrate
from scipy.spatial import distance
from scipy.signal import correlate

import mlflow
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

print(f'SciPy version: {scipy.__version__}')

In [ ]:
# Setup
spark = SparkSession.builder.appName('SciPy-Math-ML').master('local[*]').getOrCreate()
mlflow.set_tracking_uri('http://localhost:5000')
mlflow.set_experiment('spark-scipy-mathematics')

## 1. Optimización Matemática

### Optimización de funciones personalizadas

In [ ]:
# Función objetivo: Rosenbrock
def rosenbrock(x):
    return sum(100.0 * (x[1:] - x[:-1]**2)**2 + (1 - x[:-1])**2)

# Optimizar
x0 = np.array([1.3, 0.7, 0.8, 1.9, 1.2])
result = optimize.minimize(rosenbrock, x0, method='BFGS')

print('Optimización de Rosenbrock:')
print(f'Solución: {result.x}')
print(f'Valor mínimo: {result.fun:.6f}')
print(f'Iteraciones: {result.nit}')

# Log a MLflow
with mlflow.start_run(run_name='rosenbrock-optimization'):
    mlflow.log_params({'method': 'BFGS', 'dimensions': len(x0)})
    mlflow.log_metric('min_value', result.fun)
    mlflow.log_metric('iterations', result.nit)

## 2. Análisis Estadístico Distribuido

In [ ]:
# Generar datos
np.random.seed(42)
n_samples = 50000

data = {
    'normal_dist': np.random.normal(100, 15, n_samples),
    'exponential_dist': np.random.exponential(5, n_samples),
    'uniform_dist': np.random.uniform(0, 100, n_samples),
    'gamma_dist': np.random.gamma(2, 2, n_samples)
}

df = spark.createDataFrame(pd.DataFrame(data))

print(f'Dataset: {df.count()} registros')
df.show(5)

In [ ]:
# Tests estadísticos con UDF
@pandas_udf("double")
def normality_test_pvalue(col1: pd.Series) -> pd.Series:
    """Test de normalidad (Shapiro-Wilk) - retorna p-value"""
    # Usar muestra para test (Shapiro-Wilk tiene límites)
    sample = col1.sample(min(5000, len(col1)), random_state=42)
    _, p_value = stats.shapiro(sample)
    return pd.Series([p_value] * len(col1))

@pandas_udf("double")
def skewness_calc(col1: pd.Series) -> pd.Series:
    """Calcular asimetría"""
    skew = stats.skew(col1)
    return pd.Series([skew] * len(col1))

@pandas_udf("double")
def kurtosis_calc(col1: pd.Series) -> pd.Series:
    """Calcular curtosis"""
    kurt = stats.kurtosis(col1)
    return pd.Series([kurt] * len(col1))

# Aplicar tests
df_stats = df.select(
    col('normal_dist'),
    skewness_calc(col('normal_dist')).alias('skewness'),
    kurtosis_calc(col('normal_dist')).alias('kurtosis')
)

print('\nEstadísticas de la distribución normal:')
df_stats.select('skewness', 'kurtosis').distinct().show()

## 3. Álgebra Lineal Distribuida

In [ ]:
# Operaciones de álgebra lineal
def matrix_operations_example():
    # Crear matriz
    A = np.random.randn(1000, 1000)
    
    # Descomposición SVD
    U, s, Vt = linalg.svd(A, full_matrices=False)
    
    # Valores propios
    eigenvalues, eigenvectors = linalg.eig(A @ A.T)
    
    # Determinante
    det = linalg.det(A)
    
    # Norma
    norm = linalg.norm(A)
    
    print('Operaciones de Álgebra Lineal:')
    print(f'Dimensiones de A: {A.shape}')
    print(f'Valores singulares (top 5): {s[:5]}')
    print(f'Determinante: {det:.4e}')
    print(f'Norma Frobenius: {norm:.4f}')
    
    return {'det': det, 'norm': norm, 'singular_values': s[:10].tolist()}

with mlflow.start_run(run_name='linear-algebra-ops'):
    results = matrix_operations_example()
    mlflow.log_metrics({'determinant': results['det'], 'frobenius_norm': results['norm']})
    mlflow.log_param('matrix_size', '1000x1000')

## 4. Distancias y Similaridades

In [ ]:
# Calcular distancias entre vectores
from scipy.spatial.distance import cdist, pdist, squareform

# Generar vectores
vectors = np.random.randn(100, 50)

# Diferentes métricas de distancia
dist_metrics = ['euclidean', 'cosine', 'manhattan', 'chebyshev']

print('Distancias entre vectores:')
for metric in dist_metrics:
    dist_matrix = pdist(vectors, metric=metric)
    avg_dist = np.mean(dist_matrix)
    print(f'{metric:15s}: {avg_dist:.4f}')

# Usar en Spark con UDF
@pandas_udf(ArrayType(FloatType()))
def compute_pairwise_distances(features: pd.Series) -> pd.Series:
    """Calcular distancias entre vectores"""
    def calc_dist(row):
        if row is None or len(row) == 0:
            return []
        # Convertir a array
        arr = np.array(row).reshape(1, -1)
        # Calcular distancias a centros (ejemplo simple)
        centers = np.random.randn(5, arr.shape[1])
        dists = cdist(arr, centers, metric='euclidean')[0]
        return dists.tolist()
    
    return features.apply(calc_dist)

print('\n✓ UDF de distancias creada')

## 5. Integración Numérica

In [ ]:
# Integración de funciones
def gaussian(x, mu, sigma):
    return (1 / (sigma * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((x - mu) / sigma)**2)

# Integrar distribución normal
mu, sigma = 0, 1
result, error = integrate.quad(lambda x: gaussian(x, mu, sigma), -3, 3)

print(f'\nIntegración de distribución normal N(0,1) en [-3, 3]:')
print(f'Resultado: {result:.6f} (esperado: ~0.9973)')
print(f'Error estimado: {error:.2e}')

# Integración múltiple
def integrand(y, x):
    return x * y**2

result_2d = integrate.dblquad(integrand, 0, 2, 0, 1)
print(f'\nIntegración doble: {result_2d[0]:.6f}')

## 6. Funciones Especiales

In [ ]:
# Funciones especiales útiles en ML
x = np.linspace(-5, 5, 100)

# Sigmoid
sigmoid = special.expit(x)

# Log-sigmoid (más estable numéricamente)
log_sigmoid = special.log_expit(x)

# Softmax
softmax_vals = special.softmax(x)

# Gamma function
gamma_vals = special.gamma(np.linspace(0.1, 5, 50))

# Visualizar
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

axes[0, 0].plot(x, sigmoid, linewidth=2, color='steelblue')
axes[0, 0].set_title('Sigmoid Function', fontweight='bold')
axes[0, 0].grid(True, alpha=0.3)

axes[0, 1].plot(x, log_sigmoid, linewidth=2, color='coral')
axes[0, 1].set_title('Log-Sigmoid Function', fontweight='bold')
axes[0, 1].grid(True, alpha=0.3)

axes[1, 0].plot(x, softmax_vals, linewidth=2, color='green')
axes[1, 0].set_title('Softmax Function', fontweight='bold')
axes[1, 0].grid(True, alpha=0.3)

axes[1, 1].plot(np.linspace(0.1, 5, 50), gamma_vals, linewidth=2, color='purple')
axes[1, 1].set_title('Gamma Function', fontweight='bold')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('special_functions.png', dpi=150)
plt.show()

print('✓ Funciones especiales visualizadas')

## 7. Aplicación Práctica: Gradient Descent Personalizado

In [ ]:
def custom_gradient_descent(X, y, learning_rate=0.01, epochs=100):
    """Implementación de gradient descent con SciPy"""
    n_samples, n_features = X.shape
    weights = np.zeros(n_features)
    bias = 0
    
    history = {'loss': [], 'weights_norm': []}
    
    for epoch in range(epochs):
        # Predicción
        y_pred = X @ weights + bias
        
        # Loss (MSE)
        loss = np.mean((y - y_pred)**2)
        
        # Gradientes
        dw = -(2/n_samples) * X.T @ (y - y_pred)
        db = -(2/n_samples) * np.sum(y - y_pred)
        
        # Update
        weights -= learning_rate * dw
        bias -= learning_rate * db
        
        # Guardar historia
        history['loss'].append(loss)
        history['weights_norm'].append(linalg.norm(weights))
        
        if epoch % 10 == 0:
            print(f'Epoch {epoch:3d}: Loss = {loss:.6f}, ||w|| = {linalg.norm(weights):.4f}')
    
    return weights, bias, history

# Generar datos
np.random.seed(42)
X = np.random.randn(1000, 5)
true_weights = np.array([2, -1, 0.5, 3, -2])
y = X @ true_weights + np.random.randn(1000) * 0.5

# Entrenar
with mlflow.start_run(run_name='custom-gradient-descent'):
    weights, bias, history = custom_gradient_descent(X, y, learning_rate=0.01, epochs=100)
    
    mlflow.log_params({'learning_rate': 0.01, 'epochs': 100})
    mlflow.log_metric('final_loss', history['loss'][-1])
    mlflow.log_metric('weights_norm', linalg.norm(weights))
    
    print(f'\nPesos aprendidos: {weights}')
    print(f'Pesos verdaderos: {true_weights}')
    print(f'Error: {linalg.norm(weights - true_weights):.6f}')

## 8. Análisis de Correlación

In [ ]:
# Generar datos correlacionados
n = 500
x1 = np.random.randn(n)
x2 = 0.8 * x1 + 0.2 * np.random.randn(n)
x3 = -0.5 * x1 + 0.5 * np.random.randn(n)
x4 = np.random.randn(n)

# Tests de correlación
pearson_corr, pearson_p = stats.pearsonr(x1, x2)
spearman_corr, spearman_p = stats.spearmanr(x1, x2)
kendall_corr, kendall_p = stats.kendalltau(x1, x2)

print('\nAnálisis de Correlación (x1 vs x2):')
print(f'Pearson:  r = {pearson_corr:.4f}, p = {pearson_p:.4e}')
print(f'Spearman: ρ = {spearman_corr:.4f}, p = {spearman_p:.4e}')
print(f'Kendall:  τ = {kendall_corr:.4f}, p = {kendall_p:.4e}')

# Matriz de correlación
data_matrix = np.column_stack([x1, x2, x3, x4])
corr_matrix = np.corrcoef(data_matrix.T)

# Visualizar
plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, 
            xticklabels=['X1', 'X2', 'X3', 'X4'],
            yticklabels=['X1', 'X2', 'X3', 'X4'])
plt.title('Matriz de Correlación', fontweight='bold', fontsize=14)
plt.tight_layout()
plt.savefig('correlation_matrix.png', dpi=150)
plt.show()

## Conclusiones

### SciPy en ML

**Casos de Uso:**
- ✅ Optimización de hiperparámetros
- ✅ Tests estadísticos para validación
- ✅ Implementación de algoritmos custom
- ✅ Análisis numérico avanzado

**Ventajas:**
- Altamente optimizado (BLAS/LAPACK)
- Funciones estables numéricamente
- Amplia variedad de algoritmos

**Integración con Spark:**
- Usar en UDFs para operaciones complejas
- Procesar particiones con operaciones matemáticas
- Combinar con Spark ML para pipelines híbridos

### Ejercicios
1. Implementar PCA desde cero con SVD
2. Crear optimizer custom para redes neuronales
3. Implementar test de hipótesis distribuido
4. Calcular distancias de Mahalanobis en Spark
5. Implementar kernel methods con SciPy